# Deep Convolutional Neural Network for Makam Recognition
10-fold cross validation

### Library importing for file reading and preprocessing

In [25]:
import glob
import os
import sys
import json
import numpy as np
from mutagen.mp3 import MP3
import librosa
from scipy.interpolate import interp2d 
from sklearn.preprocessing import normalize
import warnings
from math import floor
import dill

### Pickle session

In [ ]:
dill.dump_session('cnn10fold.db')

In [ ]:
dill.load_session('cnn10fold.db')

### File reading using provided folds and preprocessing

In [27]:
#--------PARAMETERS--------#

#Choose truncation option
# 1 : 15 seconds starting from the start
# 2 : 15 seconds around the middle
# 3 : 15 seconds from the end
trunc_option = 2

#Choose bins and bins per octave
n_bins = 371
bins_per_octave = 53

#--------------------------#

#Ignore mp3 read warning
warnings.filterwarnings("ignore", message="PySoundFile failed. Trying audioread instead.")

makams = ["Acemasiran", "Acemkurdi", "Bestenigar", "Beyati", "Hicaz", "Hicazkar", "Huseyni", "Huzzam", "Karcigar", "Kurdilihicazkar", "Mahur", "Muhayyer", "Neva", "Nihavent", "Rast", "Saba", "Segah", "Sultaniyegah", "Suzinak", "Ussak"]

#Arrays containing all constant-Q transforms of the soundfiles per fold
X_train = [[],[],[],[],[],[],[],[],[],[]]
X_test = [[],[],[],[],[],[],[],[],[],[]]

#Array containing all makam labels per fold
y_train = [[],[],[],[],[],[],[],[],[],[]]
y_test = [[],[],[],[],[],[],[],[],[],[]]

#Import folds
f = open("./dlfm_makam_recognition_data/folds_updated.json")
folds = json.load(f)


#Compute cqts and import labels
for i in range(len(folds)): #in number of folds
    
    print("\nLoading fold " + str(i) + ".")
    
    #compute all training cqts and import train labels
    for source in range(len(folds[i][1]["training"]["sources"])):
        
        file_name = ("./soundfiles/" + folds[i][1]["training"]["sources"][source] + ".mp3")

        #get file duration for loading
        try:
            audio = MP3(file_name)
        except:
            print("Problem with:", file_name)
            continue
        audio = MP3(file_name)
        duration = audio.info.length
        
        #load 15 seconds from the mp3 according to truncation option
        if (trunc_option == 1): #15 seconds form the start
            #load soundfile
            y, sr = librosa.core.load(file_name, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
            
        if (trunc_option == 2): #15 seconds around the center
            offset = (duration-15)/2
            #load soundfile
            y, sr = librosa.core.load(file_name, offset=offset, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
            
        if (trunc_option == 3): #15 seconds from the end
            offset = duration-15
            #load soundfile
            y, sr = librosa.core.load(file_name, offset=offset, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
        
        #Normalization
        cqt = (cqt - np.mean(cqt)) / (np.std(cqt) + 1e-8)

        X_train[i].append(cqt)
        
        #import makam training label
        y_train[i].append(makams.index(folds[i][1]["training"]["modes"][source]))
        
        sys.stdout.write("\rLoading training %i/900" % source)
        sys.stdout.flush()
        
        
    #compute all testing cqts and import test labels
    for ref in range(len(folds[i][1]["testing"])):
        
        file_name = ("./soundfiles/" + folds[i][1]["testing"][ref]["source"] + ".mp3")

        #get file duration for loading
        try:
            audio = MP3(file_name)
        except:
            print("Problem with:", file_name)
            continue
        duration = audio.info.length
        
        #load 15 seconds from the mp3 according to truncation option
        if (trunc_option == 1): #15 seconds form the start
            #load soundfile
            y, sr = librosa.core.load(file_name, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
            
        if (trunc_option == 2): #15 seconds around the center
            offset = (duration-15)/2
            #load soundfile
            y, sr = librosa.core.load(file_name, offset=offset, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
            
        if (trunc_option == 3): #15 seconds from the end
            offset = duration-15
            #load soundfile
            y, sr = librosa.core.load(file_name, offset=offset, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
        
        #Normalization
        cqt = (cqt - np.mean(cqt)) / (np.std(cqt) + 1e-8)
        
        X_test[i].append(cqt)
        
        #import makam testing label
        y_test[i].append(makams.index(folds[i][1]["testing"][ref]["mode"]))
        
        sys.stdout.write("\rLoading testing %i/100" % ref)
        sys.stdout.flush()
        
print("Fold loading completed.")

print("X_train shape:", len(X_train), len(X_train[0]), X_train[0][0].shape)
print("X_test shape:", len(X_test), len(X_test[0]), X_test[0][0].shape)
print("y_train shape:", len(y_train))
print("y_test shape:", len(y_test))

f.close()

Loading fold 0.
899/100Loading fold 1.
899/100Fold loading completed.
X_train shape: 10 900 (371, 646)
X_test shape: 10 100 (371, 646)
y_train shape: 10
y_test shape: 10


### Library importing for deep learning

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


### Data preparation

In [29]:
#one-hot encode target vector
Y_train = []
Y_test = []
for i in range(len(folds)):
    Y_train.append(to_categorical(np.asarray(y_train[i])))
    Y_test.append(to_categorical(np.asarray(y_test[i])))
#channel dimension
for i in range(len(folds)):
    X_train[i] = (np.asarray(X_train[i])).reshape(len(X_train[i]), X_train[i][0].shape[0], X_train[i][0].shape[1], 1)
    X_test[i] = (np.asarray(X_test[i])).reshape(len(X_test[i]), X_test[i][0].shape[0], X_test[i][0].shape[1], 1)
print("X_train shape:", X_train[0].shape)
print("X_test shape:", X_test[0].shape)
print("y_train shape:", Y_train[0].shape)
print("y_test shape:", Y_test[0].shape)

X_train shape: (900, 371, 646, 1)
X_test shape: (100, 371, 646, 1)
y_train shape: (900, 20)
y_test shape: (100, 20)


### Constructing model

In [30]:
models = []
for i in range(len(folds)):
    #Network topology
    model = Sequential()

    #3 convolutional layers
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(X_train[i][0].shape[0], X_train[i][0].shape[1], 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(X_train[i][0].shape[0], X_train[i][0].shape[1], 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=64, kernel_size=(2,2), activation='relu', input_shape=(X_train[i][0].shape[0], X_train[i][0].shape[1], 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))

    #Output layer
    model.add(Dense(20, activation='softmax'))

    #Compile model
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.summary()
    
    models.append(model)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 369, 644, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 184, 322, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 184, 322, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 182, 320, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 91, 160, 64)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 91, 160, 64)       256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 90, 159, 64)      

### Model training

In [32]:
histories = []
for i in range(len(folds)):
    history = models[i].fit(X_train[i], Y_train[i], validation_split=0.25, epochs=5, batch_size=30)
    histories.append(history)

Train on 675 samples, validate on 225 samples
Epoch 1/10


/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


675/675 [==============================] - 245s 363ms/step - loss: 1.3738 - accuracy: 0.9035 - val_loss: 0.8469 - val_accuracy: 0.9122
Epoch 2/10
675/675 [==============================] - 233s 345ms/step - loss: 1.4614 - accuracy: 0.9021 - val_loss: 1.3819 - val_accuracy: 0.9000
Epoch 3/10
675/675 [==============================] - 218s 323ms/step - loss: 1.3254 - accuracy: 0.9064 - val_loss: 1.5142 - val_accuracy: 0.9000
Epoch 4/10
675/675 [==============================] - 221s 327ms/step - loss: 1.3997 - accuracy: 0.9064 - val_loss: 1.4850 - val_accuracy: 0.9000
Epoch 5/10
675/675 [==============================] - 222s 329ms/step - loss: 1.4231 - accuracy: 0.9055 - val_loss: 1.5337 - val_accuracy: 0.9000
Epoch 6/10
675/675 [==============================] - 256s 380ms/step - loss: 1.3517 - accuracy: 0.9079 - val_loss: 1.4355 - val_accuracy: 0.9000
Epoch 7/10
675/675 [==============================] - 295s 437ms/step - loss: 1.3582 - accuracy: 0.9075 - val_loss: 1.4378 - val_accura

/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_op.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


675/675 [==============================] - 247s 366ms/step - loss: 1.3032 - accuracy: 0.9067 - val_loss: 0.9250 - val_accuracy: 0.9187
Epoch 2/10
675/675 [==============================] - 259s 384ms/step - loss: 1.3960 - accuracy: 0.9064 - val_loss: 1.1891 - val_accuracy: 0.9044
Epoch 3/10
675/675 [==============================] - 217s 322ms/step - loss: 1.4318 - accuracy: 0.9050 - val_loss: 1.2478 - val_accuracy: 0.9102
Epoch 4/10
675/675 [==============================] - 228s 338ms/step - loss: 1.5005 - accuracy: 0.9013 - val_loss: 1.2248 - val_accuracy: 0.9200
Epoch 5/10
675/675 [==============================] - 219s 324ms/step - loss: 1.5145 - accuracy: 0.9006 - val_loss: 1.2286 - val_accuracy: 0.9196
Epoch 6/10
675/675 [==============================] - 220s 325ms/step - loss: 1.4877 - accuracy: 0.9021 - val_loss: 1.2270 - val_accuracy: 0.9200
Epoch 7/10
675/675 [==============================] - 256s 380ms/step - loss: 1.3898 - accuracy: 0.9062 - val_loss: 1.2274 - val_accura

### Model evaluation

In [35]:
scores = []
y_pred = []
for i in range(len(folds)):
    score = models[i].evaluate(X_test[i], Y_test[i], verbose=1)
    scores.append(score)
print(scores)

100/100 [==============================] - 6s 64ms/step
[[1.4257094478607177, 0.9049999117851257], [1.4570238018035888, 0.9049999117851257]]
